In [1]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [2]:
using Revise
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

include("../VARs/utils.jl")
includet("../bootstrap_model.jl")

In [3]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1473,1.8543,0.1012,0.4365,-2.9008,-0.6469,-0.1883,0.0258,2.1941,0.6874,0.1474
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1479,1.7607,0.0658,0.4053,-2.9112,-0.596,-0.2186,0.032,1.9119,0.445,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0        0.0980477
 0.0980477  1.0

In [4]:
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

ret_mean = mean(values(data), dims=1)[1,:]
ret_cov = cov(values(data), dims=1)

T = 25 * freq
n_samples = 10_000
n_assets = length(assets_names)

scenarios = zeros(n_assets, T, n_samples)

for t in 1:T 
    scenarios[:,t, :] = rand(MvNormal(ret_mean, ret_cov), n_samples)
end

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.0659,0.0186
5,0.0657,0.0182
10,0.066,0.0178
25,0.0654,0.0178


,Total_Stock_CPI,Price_10Y_CPI
1,0.1484,0.0786
5,0.0673,0.0352
10,0.0469,0.0247
25,0.0295,0.0158


,Total_Stock_CPI,Price_10Y_CPI
1,0.0094,0.0205
5,-0.0291,0.0024
10,0.003,0.0152
25,-0.0463,0.0117


,Total_Stock_CPI,Price_10Y_CPI
1,-0.0435,-0.0103
5,0.0547,-0.0496
10,-0.0016,0.0223
25,0.0164,-0.0587


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.2685,-0.2223,-0.176,-0.0343,0.067,0.1657,0.3094,0.3575,0.4145
5,-0.0927,-0.0671,-0.0467,0.0217,0.0662,0.111,0.1754,0.196,0.2243
10,-0.0461,-0.0261,-0.0106,0.0345,0.0658,0.0974,0.143,0.1585,0.1748
25,-0.0043,0.007,0.0168,0.0453,0.0654,0.0858,0.1126,0.1219,0.1329


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.163,-0.1357,-0.1104,-0.0342,0.0177,0.0719,0.1494,0.1733,0.2005
5,-0.0629,-0.0503,-0.0405,-0.0055,0.0181,0.0414,0.0763,0.0868,0.1003
10,-0.0396,-0.0299,-0.0226,0.0011,0.0176,0.0345,0.0583,0.0666,0.0755
25,-0.0187,-0.0135,-0.0084,0.0073,0.0178,0.0283,0.0437,0.0487,0.0542


In [5]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.216584  0.291093  0.349591  0.422503  0.582945

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 29.0  51.0  71.0  104.0  206.0

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.14165  0.204141  0.253141  0.319319  0.475656

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 41.0  74.0  111.0  167.25  284.0

### MVNormal based on calibration

In [12]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.year, last)

freq = 1
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1985, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1655,0.3177,0.109,0.2407,-0.4607,-1.3705,0.0402,-0.0527,2.0965,0.6585,0.1461
Price_10Y,0.0953,0.2605,0.0592,0.1252,-0.1786,-0.1749,0.0025,-0.1121,-0.1603,0.6207,0.0597


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1654,0.2908,0.0817,0.2045,-0.461,-1.2908,0.0059,-0.0458,1.6171,0.494,0.1265
Price_10Y_CPI,0.0995,0.2223,0.0319,0.1043,-0.2408,-0.3399,-0.0138,-0.1103,0.095,0.3204,0.0202


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0        0.0479353
 0.0479353  1.0

In [13]:
#.095 * .1 + .095

0.1043 * .6 + 1. * 0.4

0.46258

In [21]:
means = [.055, .01]
stds = [.175, .10]
corrm = [1. 0.048; .048 1.]

covm = cor2cov(corrm, stds)


T = 25 * freq
n_samples = 10_000
n_assets = length(assets_names)

scenarios = zeros(n_assets, T, n_samples)

for t in 1:T 
    scenarios[:,t, :] = rand(MvNormal(means, covm), n_samples)
end

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(expm1.(ret_in_years), assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(expm1.(ret_in_years[a, :, :]), [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  





,Total_Stock_CPI,Price_10Y_CPI
1,0.0734,0.0159
5,0.0603,0.0108
10,0.0584,0.0105
25,0.0573,0.0101


,Total_Stock_CPI,Price_10Y_CPI
1,0.188,0.1027
5,0.0825,0.0456
10,0.0587,0.0321
25,0.037,0.0202


,Total_Stock_CPI,Price_10Y_CPI
1,0.592,0.2973
5,0.2655,0.155
10,0.2168,0.0905
25,0.1391,0.0422


,Total_Stock_CPI,Price_10Y_CPI
1,0.6958,0.1517
5,0.0633,0.0356
10,0.1066,-0.0002
25,0.0131,0.0722


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.2909,-0.2432,-0.1993,-0.0612,0.0556,0.1888,0.4079,0.488,0.5904
5,-0.1138,-0.0903,-0.0699,0.0037,0.0563,0.1135,0.2019,0.23,0.2667
10,-0.0697,-0.0498,-0.0354,0.018,0.0562,0.0972,0.158,0.1798,0.2033
25,-0.025,-0.0129,-0.0026,0.032,0.0566,0.0823,0.1195,0.1324,0.1478


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.2015,-0.1695,-0.1444,-0.0554,0.011,0.0812,0.1928,0.2327,0.2746
5,-0.0899,-0.0756,-0.062,-0.0206,0.0092,0.0409,0.0875,0.1043,0.1234
10,-0.0612,-0.0512,-0.0411,-0.0116,0.0101,0.0317,0.0638,0.0749,0.0872
25,-0.0368,-0.0294,-0.023,-0.0037,0.0101,0.0235,0.0437,0.0504,0.0581


In [22]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.156336  0.269768  0.356358  0.459072  0.669802

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 3.0  6.0  8.0  12.0  21.0

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.131953  0.229312  0.308386  0.407233  0.599884

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 4.0  8.0  13.0  19.0  25.0

In [16]:
moving_stds = moving(std, real_returns, 20) 

display(mean(moving_stds))

display("bonds standard devations")
display(quantile(values(moving_stds)[:,1], [.03, .25, .5, .75, .97])')


display("equity standard devations")
display(quantile(values(moving_stds)[:,2], [.03, .25, .5, .75, .97])')



1×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 2024-07-01 to 2024-07-01
┌────────────┬─────────────────┬───────────────┐
│            │ Total_Stock_CPI │ Price_10Y_CPI │
├────────────┼─────────────────┼───────────────┤
│ 2024-07-01 │        0.169599 │     0.0933804 │
└────────────┴─────────────────┴───────────────┘

"bonds standard devations"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.120654  0.159893  0.175245  0.183315  0.19649

"equity standard devations"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.0587443  0.0805222  0.0947667  0.11453  0.122471